# Flightdata Preprocessor

In [3]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
import logging
import matplotlib.pyplot as plt
import sys

In [352]:
sys.path.append('./')

In [351]:
df = pd.read_csv('../assets/data.txt', sep="\t", encoding="utf-8")
del df['nix']

In [171]:
df.head()

,Flight,Date,Origin,Destination,Equipment,Flight Time,ETD,ATD,ETA,ATA
0,LH922,18-Oct-20,Frankfurt (FRA),London (LHR),A20N (D-AING),1:09,21:30,21:42,22:10,Landed 21:50
1,LH922,17-Oct-20,Frankfurt (FRA),London (LHR),A20N (D-AINL),1:03,21:30,22:04,22:10,Landed 22:08
2,LH922,16-Oct-20,Frankfurt (FRA),London (LHR),A320 (D-AIUY),1:07,21:30,21:41,22:10,Landed 21:47
3,LH922,15-Oct-20,Frankfurt (FRA),London (LHR),A20N (D-AIJB),1:08,21:30,21:39,22:10,Landed 21:48
4,LH922,14-Oct-20,Frankfurt (FRA),London (LHR),A20N (D-AINJ),1:06,21:30,21:34,22:10,Landed 21:40


In [172]:
def clean_data(df):
    """
    Clean data rows, such as invalid characters, NaN, Landed ...
    """
    _df = df.copy()
    logging.warning(f'Received df of len {len(_df)}...')
    for _ in ['Origin', 'Destination', 'Equipment']:
        _df = _df[_df[_].str.contains("\)")]
    for _ in ['Flight Time', 'ETD', 'ATD', 'ETA', 'ATA']:
        _df = _df[~(_df[_].str.contains("-")) & ~(_df[_].str.contains("—"))]
    _df = _df[(_df['ATA'].str.contains("Landed"))]
    _df['ATA'] = _df['ATA'].str.replace('Landed ','')
    _df['ATA'] = _df['ATA'].str.replace(u'Landed\xa0','')
    logging.warning(f'Returned df of len {len(_df)}...')
    return _df

In [173]:
_df = clean_data(df)

In [174]:
def clean_codes(df):
    """
    Clean airport and equipment codes
    """
    _df = df.copy()
    for _ in ['Origin', 'Destination']:
        _df[[f'{_}_City', f'{_}_Code']] = _df[_].str.split(u"\xa0\(", n = 1, expand = True)
    _df[['Equipment_Type', 'Equipment_Reg']] = _df['Equipment'].str.split(u"\xa0\(", n = 1, expand = True)
    for _ in ['Equipment_Reg', 'Origin_Code', 'Destination_Code']:
        _df[_] = _df[_].str.replace(")", "")
    return _df[['Flight', 'Date', 'Equipment_Type', 'Equipment_Reg', 'Origin_City', 'Origin_Code', 'Destination_City', 'Destination_Code', 'Flight Time', 'ETD', 'ETA', 'ATD', 'ATA']]

In [175]:
_df = clean_codes(_df)

In [201]:
def get_carrier_code(df):
    """
    Get carrier codes
    """
    _df = df.copy()
    carrier = {}
    carrier_list = []
    for index, row in _df.iterrows():
        if row['Flight'][:2] not in carrier.keys():
            carrier[row['Flight'][:2]] = ""
        carrier_list.append(row['Flight'][:2])
    return carrier, carrier_list

In [202]:
carrier, carrier_list = get_carrier_code(_df)

In [214]:
def set_carrier_info(df, carrier, carrier_class, carrier_list):
    """
    Assign airlines names + class
    """
    _df = df.copy()
    _df['Carrier_Name'] = carrier_list
    _df['Carrier_Name'] = _df['Carrier_Name'].map(carrier)
    _df['Carrier_Class'] = _df['Carrier_Name'].map(carrier_class)
    return _df

In [218]:
_df = set_carrier_info(_df, carrier, carrier_class, carrier_list)

In [208]:
carrier = {
    'LH': 'Lufthansa',
    'BA': 'British Airways',
    'LO': 'LOT',
    'SK': 'SAS',
    'UX': 'Air Europa',
    'IB': 'Iberia',
    'KL': 'KLM Royal Dutch Airlines',
    'DY': 'Norwegian',
    'ET': 'Ethiopian',
    'FR': 'Ryanair',
    'HV': 'Transavia',
    'VY': 'Vueling',
    'EI': 'Aer Lingus',
    'U2': 'EasyJet'
    }

In [217]:
carrier_class = {
    'Lufthansa': 0,
    'British Airways': 0,
    'LOT': 0,
    'SAS': 0,
    'Air Europa': 0,
    'Iberia': 0,
    'KLM Royal Dutch Airlines': 0,
    'Norwegian': 1,
    'Ethiopian': 0,
    'Ryanair': 1,
    'Transavia': 1,
    'Vueling': 1,
    'Aer Lingus': 0,
    'EasyJet': 1
    }

In [227]:
def detect_dates(df):
    """
    Detect dates and reformat it
    """
    _df = df.copy()
    date_list = []
    for index, row in _df.iterrows():
        date_list.append(datetime.strptime(row['Date'], '%d-%b-%y').date().strftime('%Y-%m-%d'))
    _df['Date'] = date_list
    return _df

In [228]:
_df = detect_dates(_df)

In [324]:
def get_time_delta(df):
    """
    Calculate time deltas
    """
    _df = df.copy()
    deltas = []
    for index, row in _df.iterrows():
        tdelta = datetime.strptime(row['ATA'], '%H:%M') - datetime.strptime(row['ETA'], '%H:%M')
        if datetime.strptime(row['ATA'], '%H:%M') < datetime.strptime(row['ETA'], '%H:%M'):
            deltas.append(-1 * int((datetime.strptime('00:00', '%H:%M') - tdelta).time().minute))
        else:
            deltas.append(int((datetime.strptime('00:00', '%H:%M') + tdelta).time().minute))
    _df['Timedelta'] = deltas
    return _df

In [325]:
_df = get_time_delta(_df)

In [333]:
def get_departure_groups(df):
    """
    Detect different departure groups throughout the day
    """
    _df = df.copy()
    dep_groups = []
    for index, row in _df.iterrows():
        if datetime.strptime(row['ETD'], '%H:%M') <= datetime.strptime('07:30', '%H:%M'):
            dep_groups.append('Red Eye')
        elif datetime.strptime(row['ETD'], '%H:%M') <= datetime.strptime('12:00', '%H:%M'):
            dep_groups.append('Morning')
        elif datetime.strptime(row['ETD'], '%H:%M') <= datetime.strptime('18:00', '%H:%M'):
            dep_groups.append('Afternoon')
        elif datetime.strptime(row['ETD'], '%H:%M') <= datetime.strptime('21:30', '%H:%M'):
            dep_groups.append('Evening')
        elif datetime.strptime(row['ETD'], '%H:%M') <= datetime.strptime('23:59', '%H:%M'):
            dep_groups.append('Night')
        else:
            dep_groups.append(None)
            logging.warning('invalid time?')
    _df['ETD_Group'] = dep_groups
    return _df

In [335]:
_df = get_departure_groups(_df)

In [115]:
def vote_majority_flight(df):
    """
    Majority vote of flights when they have an unusual leg, so that we only keep the "real" flights
    """
    _df = df.copy()
    flight_filtered = pd.DataFrame()
    flight_list = list(df.Flight.drop_duplicates())
    logging.warning(f'Loaded {len(flight_list)} flights.')
    # Loop through flight list and find major flight
    for flight in flight_list:
        _flight = _df[(_df['Flight'] == flight)].groupby(['Flight', 'Origin_City', 'Destination_City']).describe().sort_values(('Timedelta', 'count'), ascending=False).reset_index()[['Flight', 'Origin_City', 'Destination_City']].head(1)
        flight_filtered = flight_filtered.append(_flight)
    flight_filtered = flight_filtered.reset_index(drop=True).droplevel(0, axis=1)
    flight_filtered.columns = ['Flight', 'Origin_City', 'Destination_City']
    return flight_filtered


In [116]:
flight_filtered = vote_majority_flight(df_old)

In [118]:
def extract_major_flight(flight_filtered, df):
    """
    Majority vote of flights when they have an unusual leg, so that we only keep the "real" flights
    """
    _df = pd.DataFrame()
    for index, row in flight_filtered.iterrows():
        df_temp = df[(df['Flight'] == row['Flight']) & (df['Origin_City'] == row['Origin_City']) & (df['Destination_City'] == row['Destination_City'])]
        _df = _df.append(df_temp)
    return _df

In [119]:
_df = extract_major_flight(flight_filtered, df_old)

In [ ]:
_df.to_csv('../assets/data_prep_majority.txt', sep="\t", encoding="utf-8", index=False)

In [5]:
# Or if already prepared
df = pd.read_csv('../assets/data_prep_majority.txt', sep="\t", encoding="utf-8")

In [16]:
df.Destination_City.value_counts()

Amsterdam      23782
London         23703
Frankfurt      22204
Copenhagen     18996
Munich         14752
Madrid         14696
Oslo           14145
Warsaw         13085
Dublin         12843
Stockholm      10950
Barcelona       9831
Addis Ababa      735
Name: Destination_City, dtype: int64

In [12]:
df[df['Origin_City'] == 'Addis Ababa'].value_counts()

Series([], dtype: int64)

In [31]:
def correct_flighttimes(df):
    _df = df.copy()
    ATA = []
    ETA = []
    ETD = []
    ATD = []
    for index, row in _df.iterrows():
        if row['Destination_City'] in ['Dublin', 'London']:
            ATA.append((datetime.strptime(row['ATA'], '%H:%M') + timedelta(hours = 1)).strftime('%H:%M'))
            ETA.append((datetime.strptime(row['ETA'], '%H:%M') + timedelta(hours = 1)).strftime('%H:%M'))
        elif row['Destination_City'] == "Addis Ababa":
            ATA.append((datetime.strptime(row['ATA'], '%H:%M') - timedelta(hours = 1)).strftime('%H:%M'))
            ETA.append((datetime.strptime(row['ETA'], '%H:%M') - timedelta(hours = 1)).strftime('%H:%M'))
        else:
            ATA.append(row['ATA'])
            ETA.append(row['ETA'])
        if row['Origin_City'] in ['Dublin', 'London']:
            ETD.append((datetime.strptime(row['ETD'], '%H:%M') - timedelta(hours = 1)).strftime('%H:%M'))
            ATD.append((datetime.strptime(row['ATD'], '%H:%M') - timedelta(hours = 1)).strftime('%H:%M'))
        elif row['Destination_City'] == "Addis Ababa":
            ETD.append((datetime.strptime(row['ETD'], '%H:%M') + timedelta(hours = 1)).strftime('%H:%M'))
            ATD.append((datetime.strptime(row['ATD'], '%H:%M') + timedelta(hours = 1)).strftime('%H:%M'))
        else:
            ETD.append(row['ETD'])
            ATD.append(row['ATD'])
    _df['ATA'] = ATA
    _df['ETA'] = ETA
    _df['ETD'] = ETD
    _df['ATD'] = ATD
    return _df


In [34]:
_df = correct_flighttimes(df)

In [62]:
_df[[datetime.strptime(time, '%H:%M') < datetime.strptime('06:30', '%H:%M') for time in list(_df['ATA'])]]

,Flight,Date,Equipment_Type,Equipment_Reg,Origin_City,Origin_Code,Destination_City,Destination_Code,Flight Time,ETD,ETA,ATD,ATA,Carrier_Name,Carrier_Class,Timedelta,ETD_Group,black sheep
152,LH922,2020-02-23,A321,D-AISX,Frankfurt,FRA,London,LHR,1:14,21:30,23:15,22:59,00:13,Lufthansa,0,58,Evening,True
253,LH922,2019-10-25,A21N,D-AIEC,Frankfurt,FRA,London,LHR,1:40,21:30,23:10,22:20,00:00,Lufthansa,0,50,Evening,True
317,LH922,2019-08-18,A21N,D-AIEA,Frankfurt,FRA,London,LHR,1:06,21:30,23:10,23:05,00:11,Lufthansa,0,1,Evening,True
326,LH922,2019-08-09,A21N,D-AIEA,Frankfurt,FRA,London,LHR,1:26,21:30,23:10,22:47,00:12,Lufthansa,0,2,Evening,True
337,LH922,2019-07-28,A21N,D-AIEB,Frankfurt,FRA,London,LHR,1:03,21:30,23:10,22:57,00:01,Lufthansa,0,51,Evening,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178250,SK554,2019-06-14,CRJ9,ES-ACM,Amsterdam,AMS,Copenhagen,CPH,0:56,21:45,23:05,23:07,00:03,SAS,0,-2,Night,True
178378,SK554,2018-12-21,A319,OY-KBP,Amsterdam,AMS,Copenhagen,CPH,0:59,21:45,23:05,23:10,00:09,SAS,0,-56,Night,True
179521,DY4252,2020-10-16,B738,SE-RPT,Barcelona,BCN,Stockholm,ARN,3:16,20:40,0:15,20:51,00:07,Norwegian,1,-8,Evening,True
179523,DY4252,2020-10-09,B738,SE-RPS,Barcelona,BCN,Stockholm,ARN,3:02,20:40,0:15,21:18,00:20,Norwegian,1,5,Evening,True


In [59]:
_df[_df['black sheep'] == True]

,Flight,Date,Equipment_Type,Equipment_Reg,Origin_City,Origin_Code,Destination_City,Destination_Code,Flight Time,ETD,ETA,ATD,ATA,Carrier_Name,Carrier_Class,Timedelta,ETD_Group,black sheep
152,LH922,2020-02-23,A321,D-AISX,Frankfurt,FRA,London,LHR,1:14,21:30,23:15,22:59,00:13,Lufthansa,0,58,Evening,True
253,LH922,2019-10-25,A21N,D-AIEC,Frankfurt,FRA,London,LHR,1:40,21:30,23:10,22:20,00:00,Lufthansa,0,50,Evening,True
317,LH922,2019-08-18,A21N,D-AIEA,Frankfurt,FRA,London,LHR,1:06,21:30,23:10,23:05,00:11,Lufthansa,0,1,Evening,True
326,LH922,2019-08-09,A21N,D-AIEA,Frankfurt,FRA,London,LHR,1:26,21:30,23:10,22:47,00:12,Lufthansa,0,2,Evening,True
337,LH922,2019-07-28,A21N,D-AIEB,Frankfurt,FRA,London,LHR,1:03,21:30,23:10,22:57,00:01,Lufthansa,0,51,Evening,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178250,SK554,2019-06-14,CRJ9,ES-ACM,Amsterdam,AMS,Copenhagen,CPH,0:56,21:45,23:05,23:07,00:03,SAS,0,-2,Night,True
178378,SK554,2018-12-21,A319,OY-KBP,Amsterdam,AMS,Copenhagen,CPH,0:59,21:45,23:05,23:10,00:09,SAS,0,-56,Night,True
179521,DY4252,2020-10-16,B738,SE-RPT,Barcelona,BCN,Stockholm,ARN,3:16,20:40,0:15,20:51,00:07,Norwegian,1,-8,Evening,True
179523,DY4252,2020-10-09,B738,SE-RPS,Barcelona,BCN,Stockholm,ARN,3:02,20:40,0:15,21:18,00:20,Norwegian,1,5,Evening,True
